In [24]:
import pandas as pd
from collections import Counter
import re
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

In [25]:
df = pd.read_csv('../data/first_dataset_lyrics_only.csv')

In [26]:
# in der Spalttexte 'Lyrics' sind die Songtitel noch enthalten
df.head()

,genre,interpret,album,release_year,songtitle,decade,song_id,full_title,release_date_components_raw,release_date_for_display,stats_raw,lyrics
0,Pop,Adel Tawil,Lieder,2013,Immer da,2010,256740,Immer da by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}","November 8, 2013","{'unreviewed_annotations': 0, 'hot': False}",Immer da Lyrics\n\nManchmal hast du gar nichts...
1,Pop,Adel Tawil,Lieder,2013,Wenn Du liebst,2010,256742,Wenn du liebst by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}","November 8, 2013","{'unreviewed_annotations': 4, 'hot': False, 'p...",Wenn du liebst Lyrics\nGanz allein im schwarze...
2,Pop,Adel Tawil,Lieder,2013,Lieder,2010,236655,Lieder by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}","November 8, 2013","{'unreviewed_annotations': 0, 'hot': False, 'p...",Lieder Lyrics\n\nIch ging wie ein Ägypter\nHab...
3,Pop,Adel Tawil,Lieder,2013,Weinen,2010,256745,Weinen by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}","November 8, 2013","{'unreviewed_annotations': 0, 'hot': False, 'p...",Weinen Lyrics\n\nDu warst die ganze Welt für m...
4,Pop,Adel Tawil,Lieder,2013,Unter Wasser,2010,256747,Unter Wasser by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}","November 8, 2013","{'unreviewed_annotations': 0, 'hot': False}","Unter Wasser Lyrics\n\nKeine Chance, es ist vi..."


In [29]:
# in der Spalttexte 'Lyrics' sind die Songtitel + 'Lyrics\n\n' noch enthalten
df['lyrics'] = df['lyrics'].str.replace(r'^.*?Lyrics', '', regex=True)
df['lyrics'] = df['lyrics'].str.replace('^.*?\n\n|\n', ' ', regex=True)

In [30]:
df['lyrics']

0        Manchmal hast du gar nichts mehr Nur noch dei...
1        Ganz allein im schwarzen Licht wie hinter Gla...
2        Ich ging wie ein Ägypter Hab' mit Tauben gewe...
3        Du warst die ganze Welt für mich Warst das, w...
4        Keine Chance, es ist viel zu tief Keine Chanc...
                              ...                        
4537     Du wolltest deine Freiheit haben Nun stehst d...
4538     Wenn die Sonne untergeht in der Provinz, ho h...
4539     Briefe von gestern Längst schon verbrannt Mei...
4540                                                 E...
4541     Der Albtraum, dem ich entstamme, steht auf Fü...
Name: lyrics, Length: 4542, dtype: object

In [31]:
df.dtypes

genre                          object
interpret                      object
album                          object
release_year                    int64
songtitle                      object
decade                          int64
song_id                         int64
full_title                     object
release_date_components_raw    object
release_date_for_display       object
stats_raw                      object
lyrics                         object
dtype: object

In [32]:
len(df)

4542

In [33]:
df.value_counts('genre')

genre
Rap         1544
Rock        1262
Pop         1125
Schlager     611
dtype: int64

In [34]:
df.release_year.sort_values().unique()

array([1966, 1967, 1968, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977,
       1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
       1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [35]:
# lyrics in Anzahl der Wörter umwandeln
df['lyrics'] = df['lyrics'].apply(lambda x: len(x.split()))

In [36]:
# genres gruppieren
df_genre = df.groupby('genre')

# durchschnittliche Lyriclänge pro genre 
avg_lyrics_length = df_genre.mean()['lyrics']

lyrics_length = df_genre.describe()['lyrics']


print(avg_lyrics_length,"\n\n\n", lyrics_length)

# Minimum für Songlänge ist hier je nach Genre 5-6 Wörter lang?

genre
Pop         255.538667
Rap         574.736399
Rock        303.881141
Schlager    265.823241
Name: lyrics, dtype: float64 


            count        mean          std   min     25%    50%     75%  \
genre                                                                    
Pop       1125.0  255.538667   516.019022   6.0  173.00  226.0  291.00   
Rap       1544.0  574.736399  2447.040684   5.0  374.75  479.0  590.00   
Rock      1262.0  303.881141  2700.453449   6.0  162.00  211.0  267.75   
Schlager   611.0  265.823241   804.723576  32.0  158.00  204.0  248.00   

              max  
genre              
Pop       17205.0  
Rap       67616.0  
Rock      95916.0  
Schlager  15024.0  


In [346]:
## kürzester Song in dem df sind 5 Wörter; in Wirklichkeit über 100
genres = df['genre'].unique()

for genre in genres:
# Filterung nach den genres
    df_g = df[df['genre']== genre]

# kürzeste Lyric herausfinden
    shortest_lyric_title = df_g.loc[df_g['lyrics'].idxmin(), 'songtitle']
    # shortest_lyrics = df.loc[df['lyrics'].apply(len).idxmin(), 'lyrics']
    shortest_lyric_artist = df_g.loc[df_g['lyrics'].idxmin(), 'interpret']
# Anzahl der Wörter in dem Song 
    word_count_short = df[df['genre']== genre]['lyrics'].min()

# längste Lyric herausfinden
    longest_lyric_title = df_g.loc[df_g['lyrics'].idxmax(), 'songtitle']
    longest_lyric_artist = df_g.loc[df_g['lyrics'].idxmax(), 'interpret']
# Anzahl der Wörter in dem Song 
    word_count_long = df[df['genre']== genre]['lyrics'].max()

    print(f"Der kürzeste {genre}-Song ist '{shortest_lyric_title}' von {shortest_lyric_artist} mit insgesamt {word_count_short} Wörtern")
    print(f"Der längste {genre}-Song ist '{longest_lyric_title}' von {longest_lyric_artist} mit insgesamt {word_count_long} Wörtern\n")
   


Der kürzeste Pop-Song ist 'Superlude' von Frittenbude mit insgesamt 6 Wörtern
Der längste Pop-Song ist 'Lass es regnen' von Rosenstolz mit insgesamt 17205 Wörtern

Der kürzeste Rap-Song ist 'Günther' von Deichkind mit insgesamt 5 Wörtern
Der längste Rap-Song ist 'Gold sinkt nach unten' von Prinz Pi mit insgesamt 67616 Wörtern

Der kürzeste Rock-Song ist 'Bullenschwein' von Die Ärzte mit insgesamt 6 Wörtern
Der längste Rock-Song ist 'Trick 17 m.S.' von Die Ärzte mit insgesamt 95916 Wörtern

Der kürzeste Schlager-Song ist 'Gloria Halleluja Amen' von Nina Hagen mit insgesamt 32 Wörtern
Der längste Schlager-Song ist 'Keine Lust auf gar nichts' von Wolfgang Petry mit insgesamt 15024 Wörtern



In [336]:
# Aufrufen der Lyrics für den kürzesten Song
df[df['songtitle'] == 'Günther']['lyrics']

1537    Telefon DialogYou might also likeEmbed
Name: lyrics, dtype: object

In [168]:
pd.options.plotting.backend = "plotly"

In [169]:
lyric_fig = avg_lyrics_length.plot(kind='bar', title='lyric length')
lyric_fig.show()

In [22]:
# Filtere das DataFrame, um nur das Genre "Pop" zu betrachten
pop_data = df[df['genre'] == 'Pop']

# Verbinde alle Lyrics-Texte zu einem einzigen langen Text
lyrics_text = ' '.join(pop_data['lyrics'].astype(str).tolist())

# Entferne Satzzeichen, Sonderzeichen und Zahlen aus dem Text
lyrics_text = re.sub(r'[^\w\s]', '', lyrics_text)
lyrics_text = re.sub(r'\d+', '', lyrics_text)

# Konvertiere den Text in eine Liste von Wörtern
words = lyrics_text.lower().split()

stop_words = set(stopwords.words('german'))
filtered_words = [w for w in words if not w in stop_words]


# Zähle die Häufigkeit der Wörter im Text
word_counts = Counter(filtered_words)

top_words = word_counts.most_common(20)
top_words_dict = {}
for word, count in top_words:
    top_words_dict[word] = count
    print(f"{word}:  {count}")



# Idee: Wie deutsch ist deutscher Pop wirklich?
## Anzahl der englischen Begriffe in deutschen Pop-Songs

# da unter den top 5 auftretenden Wörtern sind kann es auch möglich sein, dass englische Texte unter den deutschen songs versteckt mit drin sin

mehr:  1354
immer:  1149
might:  1121
you:  889
leben:  797
mal:  749
schon:  741
nie:  704
zeit:  704
weiß:  651
welt:  617
liebe:  611
geht:  587
hast:  543
likeembed:  530
ganz:  513
nacht:  509
tag:  509
gut:  507
lass:  501


In [23]:
# top_words_dg = pd.DataFrame(top_words_dict)
# top_word_fig = top_words_dg.plot(kind='bar', title='lyric length')
# top_word_fig.show()